## 05. 클래스로 파이토치 모델 구현하기

파이토치의 대부분의 구현체들은 대부분 모델을 생성할 때 클래스(Class)를 사용하고 있다. 앞서 배운 선형 회귀를 클래스로 구현해보겠다. 앞서 구현한 코드와 다른 점은 오직 클래스로 모델을 구현했다는 점이다.



### 1. 모델을 클래스로 구현하기

앞서 단순 선형 회귀 모델은 다음과 같이 구현했었다.

``` python
# 모델을 선언 및 초기화. 단순 선형 회귀이므로 input_dim=1, output_dim=1.
model = nn.Linear(1,1)
```

이를 클래스로 구현하면 다음과 같다.




In [2]:
import torch.nn as nn

class LinearRegressionModel(nn.Module):  # torch.nn.Module을 상속받는 파이썬 클래스

    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1) # 단순 선형 회귀이므로 input_dim = 1, output_dim = 1.

    def forward(self, x):
        return self.linear(x)

In [3]:
model = LinearRegressionModel()
model

LinearRegressionModel(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)

위와 같은 클래스를 사용한 모델 구현 형식은 대부분의 파이토치 구현체에서 사용하고 있는 방식으로 반드시 숙지할 필요가 있다.

클래스(class) 형태의 모델은 nn.Module을 상속받는다. 그리고 __init__()에서 모델의 구조와 동작을 정의하는 생성자를 정의한다. 이는 파이썬에서 객체가 갖는 속성값을 초기화하는 역할로, 객체가 생성될 때 자동으로 호출된다. super() 함수를 부르면 여기서 만든 클래스는 nn.Module 클래스의 속성들을 가지고 초기화 된다. forward() 함수는 모델이 학습데이터를 입력받아서 forward 연산을 진행시키는 함수이다. 이 forward() 함수는 model 객체를 데이터와 함께 호출하면 자동으로 실행이 된다. 예를 들어 model이란 이름의 객체를 생성 후, model(입력 데이터)와 같은 형식으로 객체를 호출하면 자동으로 forward 연산이 수행된다.

* $H(x)$ 식에 입력 $x$로부터 예측된 $y$를 얻는 것을 forward 연산이라고 한다.

앞서 다중 선형 회귀 모델은 다음과 같이 구현했었다.

``` python
# 모델을 선언 및 초기화. 다중 선형 회귀이므로 input_dim=3, output_dim=1.
model = nn.Linear(3,1)
```

이를 클래스로 구현하면 다음과 같다.


In [4]:
class MultivariateLinearRegressionModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1) # 다중 선형 회귀이므로 input_dim = 3, output_dim = 1.

    def forward(self, x):
        return self.linear(x)

In [5]:
model = MultivariateLinearRegressionModel()
model

MultivariateLinearRegressionModel(
  (linear): Linear(in_features=3, out_features=1, bias=True)
)

### 2. 단순 선형 회귀 클래스로 구현하기

이제 모델을 클래스로 구현한 코드를 보겠다. 달라진 점은 모델을 클래스로 구현했다는 점 뿐이다. 다른 코드는 전부 동일하다.

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
torch.manual_seed(1)

In [8]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])


In [9]:
class LinearRegressionModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

In [10]:
model = LinearRegressionModel()
model

LinearRegressionModel(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)

In [12]:
# optimizer 설정. 경사 하강법 SGD를 사용하고 learning rate를 의미하는 lr은 0.01
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [13]:
# 전체 훈련 데이터에 대해 경사 하강법을 2000회 반복
nb_epochs = 2000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train) # <== 파이토치에서 제공하는 평균 제곱 오차 함수

    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()

    # 비용 함수를 미분하여 gradient 계산
    cost.backward() # backward 연산

    # W와 b를 업데이트
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/2000 Cost: 13.103541
Epoch  100/2000 Cost: 0.002791
Epoch  200/2000 Cost: 0.001724
Epoch  300/2000 Cost: 0.001066
Epoch  400/2000 Cost: 0.000658
Epoch  500/2000 Cost: 0.000407
Epoch  600/2000 Cost: 0.000251
Epoch  700/2000 Cost: 0.000155
Epoch  800/2000 Cost: 0.000096
Epoch  900/2000 Cost: 0.000059
Epoch 1000/2000 Cost: 0.000037
Epoch 1100/2000 Cost: 0.000023
Epoch 1200/2000 Cost: 0.000014
Epoch 1300/2000 Cost: 0.000009
Epoch 1400/2000 Cost: 0.000005
Epoch 1500/2000 Cost: 0.000003
Epoch 1600/2000 Cost: 0.000002
Epoch 1700/2000 Cost: 0.000001
Epoch 1800/2000 Cost: 0.000001
Epoch 1900/2000 Cost: 0.000000
Epoch 2000/2000 Cost: 0.000000


### 3. 다중 선형 회귀 클래스로 구현하기

이제 모델을 클래스로 구현한 코드를 보겠다. 달라진 점은 모델을 클래스로 구현했다는 점 뿐이다. 다른 코드는 전부 동일하다.

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [23]:
torch.manual_seed(1)

In [24]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [25]:
class MultivariateLinearRegressionModel(nn.Module):

    def __init__(self):

        super().__init__()
        self.linear = nn.Linear(3, 1) # 다중 선형 회귀이므로 input_dim = 3, output_dim = 1.

    def forward(self, x):
        return self.linear(x)


In [26]:
model = MultivariateLinearRegressionModel()
model

MultivariateLinearRegressionModel(
  (linear): Linear(in_features=3, out_features=1, bias=True)
)

In [27]:
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-5)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 1e-05
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [28]:
nb_epochs = 2000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)
    # model(x_train)은 model.forward(x_train)와 동일함.

    # cost 계산
    cost = F.mse_loss(prediction, y_train) # <== 파이토치에서 제공하는 평균 제곱 오차 함수

    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    
    # 비용 함수를 미분하여 gradient 계산
    cost.backward()

    # W와 b를 업데이트
    optimizer.step()

    if epoch % 100 == 0:
        # 100번마다 로그 출력
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/2000 Cost: 31667.597656
Epoch  100/2000 Cost: 0.225993
Epoch  200/2000 Cost: 0.223911
Epoch  300/2000 Cost: 0.221941
Epoch  400/2000 Cost: 0.220059
Epoch  500/2000 Cost: 0.218271
Epoch  600/2000 Cost: 0.216575
Epoch  700/2000 Cost: 0.214950
Epoch  800/2000 Cost: 0.213413
Epoch  900/2000 Cost: 0.211952
Epoch 1000/2000 Cost: 0.210560
Epoch 1100/2000 Cost: 0.209232
Epoch 1200/2000 Cost: 0.207967
Epoch 1300/2000 Cost: 0.206761
Epoch 1400/2000 Cost: 0.205619
Epoch 1500/2000 Cost: 0.204522
Epoch 1600/2000 Cost: 0.203484
Epoch 1700/2000 Cost: 0.202485
Epoch 1800/2000 Cost: 0.201542
Epoch 1900/2000 Cost: 0.200635
Epoch 2000/2000 Cost: 0.199769
